# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [65]:
import numpy as np
import pandas as pd

# Challenge 1 - Independent Sample T-tests

In this challenge, we will be using the Pokemon dataset. Before applying statistical methods to this data, let's first examine the data.

To load the data, run the code below.

In [66]:
pokemon = pd.read_csv('../pokemon.csv',index_col=0)

Let's start off by looking at the `head` function in the cell below.

In [67]:
pokemon.head()

,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
#,,,,,,,,,,,,
1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


The first thing we would like to do is compare the legendary Pokemon to the regular Pokemon. To do this, we should examine the data further. What is the count of legendary vs. non legendary Pokemons?

In [68]:
pokemon['Legendary'].value_counts()

False    735
True      65
Name: Legendary, dtype: int64

Compute the mean and standard deviation of the total points for both legendary and non-legendary Pokemon.

In [69]:
pokemon.groupby('Legendary').agg({'Total':[np.mean,np.std]})

Total            
                 mean         std
Legendary                        
False      417.213605  106.760417
True       637.384615   60.937389

The computation of the mean might give us a clue regarding how the statistical test may turn out; However, it certainly does not prove whether there is a significant difference between the two groups.

In the cell below, use the `ttest_ind` function in `scipy.stats` to compare the the total points for legendary and non-legendary Pokemon. Since we do not have any information about the population, assume the variances are not equal.

In [70]:
from scipy.stats import ttest_ind

stat, pvalue = ttest_ind(pokemon[pokemon['Legendary']==False]['Total'],
                         pokemon[pokemon['Legendary']==True]['Total'],
                         equal_var=False)
print(f'p-value: {pvalue}')

p-value: 9.357954335957446e-47


What do you conclude from this test? Write your conclusions below.

__`As the pvalue tends zero, the samples are different from each other concluding that the total points between the legendary and non-legendary pokemons are very different`__

How about we try to compare the different types of pokemon? In the cell below, list the types of Pokemon from column `Type 1` and the count of each type.

In [71]:
pokemon['Type 1'].value_counts()

Water       112
Normal       98
Grass        70
Bug          69
Psychic      57
Fire         52
Electric     44
Rock         44
Ghost        32
Dragon       32
Ground       32
Dark         31
Poison       28
Steel        27
Fighting     27
Ice          24
Fairy        17
Flying        4
Name: Type 1, dtype: int64

Since water is the largest group of Pokemon, compare the mean and standard deviation of water Pokemon to all other Pokemon.

In [72]:
water_mean, water_std = np.mean(pokemon[pokemon['Type 1']== 'Water']['Total']), np.std(pokemon[pokemon['Type 1']== 'Water']['Total'])
non_water_mean, non_water_std = np.mean(pokemon[pokemon['Type 1']!= 'Water']['Total']), np.std(pokemon[pokemon['Type 1']!= 'Water']['Total'])
print(f'''Water pokemon mean and std: {round(water_mean,3)}, {round(water_std,3)}
Non-water pokemon mean and std: {round(non_water_mean,3)}, {round(non_water_std,3)}
''')

Water pokemon mean and std: 430.455, 112.682
Non-water pokemon mean and std: 435.859, 121.004



Perform a hypothesis test comparing the mean of total points for water Pokemon to all non-water Pokemon. Assume the variances are equal. 

In [73]:
stat, pvalue = ttest_ind(pokemon[pokemon['Type 1']== 'Water']['Total'],
                         pokemon[pokemon['Type 1']!= 'Water']['Total'],
                         equal_var=True)
print(f'pvalue: {pvalue}')

pvalue: 0.6587140317488793


Write your conclusion below.

__`As the pvalue is greater than alpha = 5% we can assume that ther is no signigicant difference between the total points of water and non-water pokemons`__

# Challenge 2 - Matched Pairs Test

In this challenge we will compare dependent samples of data describing our Pokemon. Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. In the cell below, import the `ttest_rel` function from `scipy.stats` and compare the two columns to see if there is a statistically significant difference between them.

In [74]:
from scipy.stats import ttest_rel
stat, pvalue = ttest_rel(pokemon['Attack'],pokemon['Defense'])
print(f'pvalue: {pvalue}')

pvalue: 1.7140303479358558e-05


Describe the results of the test in the cell below.

__`We can conclude that the attack and defense are very different from each other because the pvalue is les than 5%`__

We are also curious about whether therer is a significant difference between the mean of special defense and the mean of special attack. Perform the hypothesis test in the cell below. 

In [75]:
stat, pvalue = ttest_rel(pokemon['Sp. Atk'],pokemon['Sp. Def'])
print(f'pvalue: {pvalue}')

pvalue: 0.3933685997548122


Describe the results of the test in the cell below.

__`Differently from the normal attack and defense the special attack and defense does not have significantly difference as the pvalue is highter than 5%`__

As you may recall, a two sample matched pairs test can also be expressed as a one sample test of the difference between the two dependent columns.

Import the `ttest_1samp` function and perform a one sample t-test of the difference between defense and attack. Test the hypothesis that the difference between the means is zero. Confirm that the results of the test are the same.

In [76]:
from scipy.stats import ttest_1samp
stat, pvalue = ttest_1samp(pokemon['Attack']-pokemon['Defense'],0)
print(f'pvalue: {pvalue}')

pvalue: 1.7140303479358558e-05


# Bonus Challenge - The Chi-Square Test

The Chi-Square test is used to determine whether there is a statistically significant difference in frequencies. In other words, we are testing whether there is a relationship between categorical variables or rather when the variables are independent. This test is an alternative to Fisher's exact test and is used in scenarios where sample sizes are larger. However, with a large enough sample size, both tests produce similar results. Read more about the Chi Squared test [here](https://en.wikipedia.org/wiki/Chi-squared_test).

In the cell below, create a contingency table using `pd.crosstab` comparing whether a Pokemon is legenadary or not and whether the Type 1 of a Pokemon is water or not.

In [77]:
tab = pd.crosstab(index=[pokemon['Legendary']], columns=np.where(pokemon['Type 1']=='Water','Water','Other'))
tab

col_0,Other,Water
Legendary,,
False,627,108
True,61,4


Perform a chi-squared test using the `chi2_contingency` function in `scipy.stats`. You can read the documentation of the function [here](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html).

In [78]:
from scipy.stats import chi2_contingency
chi2_contingency(tab)

(2.9429200762850503,
 0.08625467249550949,
 1,
 array([[632.1, 102.9],
        [ 55.9,   9.1]]))

Based on a 95% confidence, should we reject the null hypothesis?

In [79]:
print(f'pvalue: {chi2_contingency(tab)[1]}')

pvalue: 0.08625467249550949


__`The pvalue is greater than 0.05 so the null hypothesis is rejected; there is no relation between water pokemon and legendarity of a pokemon`__